## 주의 사항 
- `pyodide_http`, `patch_all`을 통해 기존 패키지를 pyodide에 맞게 패치한다. 
- "CORS unlock" 크롬 엑스텐션을 쓴다. 

## Source 
https://ponder.io/professional-pandas-the-pandas-assign-method-and-chaining/

## TL; DR
- assign을 활용한 데이터프레임 변형 체이닝 

In [16]:
%pip -q install pyodide-http
import pandas as pd 
import pyodide_http 
pyodide_http.patch_all()  # Patch all libraries

In [13]:
raw = pd.read_csv('https://github.com/mattharrison/datasets/raw/master/data/alta-noaa-1980-2019.csv', parse_dates=['DATE'])
def tweak_alta(df):
    return (df
            .loc[:, ['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 
                     'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS']]
           )
alta = tweak_alta(raw)
alta

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS
0,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-01,0.10,2.0,29.0,38.0,25.0,25.0
1,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-02,0.43,3.0,34.0,27.0,18.0,18.0
2,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-03,0.09,1.0,30.0,27.0,12.0,18.0
3,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-04,0.00,0.0,30.0,31.0,18.0,27.0
4,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-05,0.00,0.0,30.0,34.0,26.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14155,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,2019-09-03,0.01,0.0,0.0,74.0,57.0,73.0
14156,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,2019-09-04,0.00,0.0,0.0,77.0,52.0,74.0
14157,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,2019-09-05,0.00,0.0,0.0,76.0,54.0,65.0
14158,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,2019-09-06,0.07,0.0,0.0,66.0,52.0,60.0


In [15]:
(alta
 .assign(PRCP_CM=alta.PRCP * 2.54)
)

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,PRCP_CM
0,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-01,0.10,2.0,29.0,38.0,25.0,25.0,0.2540
1,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-02,0.43,3.0,34.0,27.0,18.0,18.0,1.0922
2,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-03,0.09,1.0,30.0,27.0,12.0,18.0,0.2286
3,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-04,0.00,0.0,30.0,31.0,18.0,27.0,0.0000
4,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-05,0.00,0.0,30.0,34.0,26.0,34.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14155,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,2019-09-03,0.01,0.0,0.0,74.0,57.0,73.0,0.0254
14156,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,2019-09-04,0.00,0.0,0.0,77.0,52.0,74.0,0.0000
14157,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,2019-09-05,0.00,0.0,0.0,76.0,54.0,65.0,0.0000
14158,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,2019-09-06,0.07,0.0,0.0,66.0,52.0,60.0,0.1778


In [17]:
def monthly_summary(df):
    my_cols =  ['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 
               'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS']
    return (df
            [my_cols]
            .groupby(pd.Grouper(key='DATE', freq='W'))
            .agg({'PRCP': 'sum', 'TMAX': 'max', 'TMIN': 'min', 'SNOW': 'sum', 'SNWD': 'mean'})
            .reset_index()
            .assign(LOCATION='Alta', 
                    SEASON='Ski')
            .assign(LOCATION=lambda df_: df_.LOCATION.where(
                                                            df_.DATE.dt.year >= 2000
                                                            ,'Alta Ski Resort'),
                    T_RANGE=lambda df_: df_.TMAX - df_.TMIN,
                    SEASON=lambda df_:(df_
                                       .SEASON
                                       .where(
                     (df_.DATE.dt.month <= 4) | (df_.DATE.dt.month >= 11), 'Summer')
                                       .add(' ')
                                       .add(df_
                                            .DATE.dt.year.astype(str)
                                            .where(df_.DATE.dt.month > 4, 
                                                   (df_.DATE.dt.year - 1).astype(str))))
                   )
           )

monthly_summary(alta)

,DATE,PRCP,TMAX,TMIN,SNOW,SNWD,LOCATION,SEASON,T_RANGE
0,1980-01-06,0.68,42.0,12.0,7.0,30.500000,Alta Ski Resort,Ski 1979,30.0
1,1980-01-13,8.76,33.0,-5.0,63.0,48.142857,Alta Ski Resort,Ski 1979,38.0
2,1980-01-20,4.68,40.0,15.0,34.0,63.285714,Alta Ski Resort,Ski 1979,25.0
3,1980-01-27,0.00,43.0,10.0,0.0,57.000000,Alta Ski Resort,Ski 1979,33.0
4,1980-02-03,2.58,38.0,-6.0,40.0,70.571429,Alta Ski Resort,Ski 1979,44.0
...,...,...,...,...,...,...,...,...,...
2066,2019-08-11,1.09,78.0,42.0,0.0,0.000000,Alta,Summer 2019,36.0
2067,2019-08-18,0.00,75.0,42.0,0.0,0.000000,Alta,Summer 2019,33.0
2068,2019-08-25,0.00,76.0,45.0,0.0,0.000000,Alta,Summer 2019,31.0
2069,2019-09-01,0.02,78.0,41.0,0.0,0.000000,Alta,Summer 2019,37.0
